<a href="https://colab.research.google.com/github/somilasthana/MachineLearningSkills/blob/master/Scratch_SOM_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
"""
Theory:
Iy is a type of neural network that is trained using unsupervised learning 
on unlabeled data. In contrast to a standard neural network a SOM only has 
input and output layer (no hidden layer in middle, but rather a SOM layer).

SOM does not need a target output, and is trained using competetive learning in 
which the neurons (weights asscociated with nodes in output layer) compete for 
activation (not minimizing error using backpropogation).



"""

Algorithm to implement Self Organizing Map
1. Randomly initialize each output node's weights, $(w_{1}^{0},w_{2}^{0},\ldots,w_{n}^{0}).$
2. Choose data point at random from input data, $x_{i}^{*}.$
3.  Find node with associated weight vector which is closest to $x_{i}^{*}.$ use euclidean distance to measure $w_{m}$
4. Determine the neighbouring vectors around the winning neuron 

      $N_{t}$=$N_{0}$ * exp( -t / ($N_{0}$/total_iterations) )    
       
      t is iteration number, 
      
      
       
5. Weights are adjusted so that the winning node and its neighbours become closer to the input $x_{i}^{*}.$ 

    $w_{i+1}^{t} = w_{t} ^{t}+ phi(t, w_{i})* L(t)*(x_{i} - w_{i}^{t})$
    
    where $L(t) = L(0) * exp(-t/totaliterations).$
    
     $phi(t, w_{i}) = exp(dist(w_{m}, w_{i})/2*N_{t}^2)$ 
     
     where $w_{m}$ is the winner node from 3.
    
   

In [1]:
# Get Data
!wget https://raw.githubusercontent.com/hammadshaikhha/Math-of-Machine-Learning-Course-by-Siraj/master/Self%20Organizing%20Maps%20for%20Data%20Visualization/Grade1Students.csv

--2019-06-20 22:45:19--  https://raw.githubusercontent.com/hammadshaikhha/Math-of-Machine-Learning-Course-by-Siraj/master/Self%20Organizing%20Maps%20for%20Data%20Visualization/Grade1Students.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 112647 (110K) [text/plain]
Saving to: ‘Grade1Students.csv’

Grade1Students.csv  100%[===================>] 110.01K  --.-KB/s    in 0.04s   

2019-06-20 22:45:19 (2.88 MB/s) - ‘Grade1Students.csv’ saved [112647/112647]



In [0]:
# Import dependencies
# numpy for matrix algbera
import numpy as np
# Pandas for data manipulation
import pandas as pd
# matplotlib for data visualization
import matplotlib.pyplot as plt


In [0]:
np.random.seed(42)

In [0]:
edu_data = pd.read_csv("/content/Grade1Students.csv")

In [5]:
edu_data.head()

,g1freelunch,g1absent,g1readscore,g1mathscore,g1listeningscore,g1wordscore
0,1,9,516,578,601,493
1,0,12,451,507,584,436
2,1,4,483,526,529,486
3,1,15,516,505,556,536
4,1,2,433,463,504,426


In [28]:
edu_data.describe()

,g1freelunch,g1absent,g1readscore,g1mathscore,g1listeningscore,g1wordscore
count,5550.000000,5550.000000,5550.000000,5550.000000,5550.000000,5550.000000
mean,0.500901,7.421261,521.307207,531.456216,567.824324,514.643063
std,0.500044,7.004582,55.278448,43.151113,33.562973,52.858396
min,0.000000,0.000000,404.000000,404.000000,477.000000,317.000000
25%,0.000000,2.000000,478.000000,502.000000,543.000000,475.000000
50%,1.000000,6.000000,516.000000,529.000000,565.000000,514.000000
75%,1.000000,10.000000,558.000000,562.000000,588.000000,551.000000
max,1.000000,84.000000,651.000000,676.000000,708.000000,601.000000


In [0]:
# Max Min Scaling
edu_data = (edu_data - edu_data.min()) / (edu_data.max() - edu_data.min())

In [30]:
edu_data.head()

,g1freelunch,g1absent,g1readscore,g1mathscore,g1listeningscore,g1wordscore
0,1.0,0.107143,0.453441,0.639706,0.536797,0.619718
1,0.0,0.142857,0.190283,0.378676,0.463203,0.419014
2,1.0,0.047619,0.319838,0.448529,0.225108,0.595070
3,1.0,0.178571,0.453441,0.371324,0.341991,0.771127
4,1.0,0.023810,0.117409,0.216912,0.116883,0.383803


In [0]:
total_num_iterations = 3*len(edu_data.index)
nodes_num = 3
input_dim = len(edu_data.columns)
learn_init = 0.1

In [0]:
# Step 1: Initialize the weight vectors 
w = 4 * np.random.rand(input_dim, nodes_num) -2

In [42]:
w.shape

(6, 3)

In [43]:
w

array([[ 1.33617766, -0.08133127,  0.34545002],
       [ 1.49501517, -0.76472435,  0.80279728],
       [ 1.44666903, -1.1895372 , -0.57408734],
       [ 1.9192407 ,  1.93287426, -1.10708028],
       [-1.73421664, -0.91631355, -0.40907217],
       [-0.2359776 ,  0.47522523,  1.59801104]])

In [0]:

for iteration in range(total_num_iterations):
  
  dist_bmu = float("inf")
  
  # Step 2: Choose data point at random from input data
  row_index = np.random.randint(len(edu_data.index))
  
  # get data point
  data_point = edu_data.loc[[row_index]]
  
  # Step 3: Find the weight vector that is closest to chosen point
  for node in range(nodes_num):
    
    # Compute euclidean distance from weight vector to chosen point
    dist_neuron = np.linalg.norm(data_point - w[:, node])
    
    if dist_neuron < dist_bmu:
      dist_bmu = dist_neuron
      w_bmu = w[:, node]
      index_bmu = node
      
  # Step 4: Define radius of winning neuron neighbourhood 
  
  learning_rate = learn_init * np.exp(-iteration/total_num_iterations)
  
  # Step 5: Update weight vectors (w_{t+1} = w_{t} + L(t)*(x_{i} - w_{t}))
  
  w[:,index_bmu] = np.add(w_bmu, learning_rate*np.subtract(data_point,w_bmu))
  

In [49]:
w

array([[ 1.33617766, -0.08133127,  0.59974042],
       [ 1.49501517, -0.76472435,  0.07519943],
       [ 1.44666903, -1.1895372 ,  0.45666219],
       [ 1.9192407 ,  1.93287426,  0.46285562],
       [-1.73421664, -0.91631355,  0.4140339 ],
       [-0.2359776 ,  0.47522523,  0.67566446]])

In [0]:
group = np.zeros(len(edu_data.index))

for index, data in edu_data.iterrows():
  
  dist_cluster = float("inf")
  
  for centroid in range(nodes_num):
    dist_centroid = np.linalg.norm(data - w[:, centroid])
    
    if dist_centroid < dist_cluster:
      dist_cluster = dist_centroid
      group[index] = centroid + 1
      
edu_data["group"] = group

In [38]:
edu_data.head()

,g1freelunch,g1absent,g1readscore,g1mathscore,g1listeningscore,g1wordscore,group
0,1.0,0.107143,0.453441,0.639706,0.536797,0.619718,1.0
1,0.0,0.142857,0.190283,0.378676,0.463203,0.419014,1.0
2,1.0,0.047619,0.319838,0.448529,0.225108,0.595070,1.0
3,1.0,0.178571,0.453441,0.371324,0.341991,0.771127,1.0
4,1.0,0.023810,0.117409,0.216912,0.116883,0.383803,1.0


In [40]:
edu_data[edu_data.group == 1].describe()

,g1freelunch,g1absent,g1readscore,g1mathscore,g1listeningscore,g1wordscore,group
count,5550.000000,5550.000000,5550.000000,5550.000000,5550.000000,5550.000000,5550.0
mean,0.500901,0.088348,0.474928,0.468589,0.393179,0.695926,1.0
std,0.500044,0.083388,0.223799,0.158644,0.145294,0.186121,0.0
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
25%,0.000000,0.023810,0.299595,0.360294,0.285714,0.556338,1.0
50%,1.000000,0.071429,0.453441,0.459559,0.380952,0.693662,1.0
75%,1.000000,0.119048,0.623482,0.580882,0.480519,0.823944,1.0
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0
